In [1]:
import pandas as pd

df = pd.read_csv('./dataset_wikidata_works.csv')
df.head()

,movement,movementLabel,artist,artistLabel,works,worksLabel,symbols,symbolsLabel,iconclass,genre,genreLabel
0,http://www.wikidata.org/entity/Q4692,Renaissance,http://www.wikidata.org/entity/Q762,Leonardo da Vinci,http://www.wikidata.org/entity/Q12418,Mona Lisa,http://www.wikidata.org/entity/Q527,sky,NaN,http://www.wikidata.org/entity/Q134307,portrait
1,http://www.wikidata.org/entity/Q4692,Renaissance,http://www.wikidata.org/entity/Q762,Leonardo da Vinci,http://www.wikidata.org/entity/Q12418,Mona Lisa,http://www.wikidata.org/entity/Q8502,mountain,NaN,http://www.wikidata.org/entity/Q134307,portrait
2,http://www.wikidata.org/entity/Q4692,Renaissance,http://www.wikidata.org/entity/Q762,Leonardo da Vinci,http://www.wikidata.org/entity/Q12418,Mona Lisa,http://www.wikidata.org/entity/Q12280,bridge,NaN,http://www.wikidata.org/entity/Q134307,portrait
3,http://www.wikidata.org/entity/Q4692,Renaissance,http://www.wikidata.org/entity/Q762,Leonardo da Vinci,http://www.wikidata.org/entity/Q12418,Mona Lisa,http://www.wikidata.org/entity/Q15324,body of water,NaN,http://www.wikidata.org/entity/Q134307,portrait
4,http://www.wikidata.org/entity/Q4692,Renaissance,http://www.wikidata.org/entity/Q762,Leonardo da Vinci,http://www.wikidata.org/entity/Q12418,Mona Lisa,http://www.wikidata.org/entity/Q107425,landscape,NaN,http://www.wikidata.org/entity/Q134307,portrait


In [18]:
#api test
import requests as rq
import json

request = rq.get('https://iconclass.org/%s.json'%('95A(SISYPHUS)6'))
json_data = request.json()
json_data['txt']['en']

'suffering, misfortune of Sisyphus'

In [31]:
# to resolve the iconclass id to natural language

iconclass = df['iconclass'].fillna('')
print(iconclass)

def iconclass_resolution(iconclass_series):
    ids = dict()
    natural_series = list()
    for i in iconclass_series:
        if i == '':
            natural_series.append('empty')
        elif i in ids:
            natural_series.append(ids[i])
        else:
            try:
                request = rq.get('https://iconclass.org/%s.json'%(i))
                json_data = request.json()
                text = json_data['txt']['en']
                print(text)
                natural_series.append(text)
                ids[i] = text
            except:
                print(f'{i} this code is not working')
    return pd.Series(natural_series)

iconclass_resolution(iconclass_series=iconclass)

0         
1         
2         
3         
4         
        ..
33992     
33993     
33994     
33995     
33996     
Name: iconclass, Length: 33997, dtype: object
adoration of the Christ-child by Mary
horse
saints
the conversion of St. Eustace: while hunting (on horseback) he is confronted with a stag bearing a crucifix between its antlers; Eustace is shown kneeling before the stag (that appears on a rock)
hoofed animals: deer
rodents: hare
border, ribbon, braid
hunting dogs
spur
adult woman
St. George and the dragon: dressed in armour and mounted on a (white) horse he brandishes his sword (his lance lies already broken on the ground); the princess nearby may be shown praying or fleeing
Christ-child looking or turning to Mary ~ Madonna-representations
the Annunciation: Mary sitting
baptism of Christ in the river Jordan: John the Baptist pouring out water on Christ's head: the Holy Ghost descends
Tobias and the angel Raphael travelling, accompanied by Tobias' dog
forest, wood
a woma

0        empty
1        empty
2        empty
3        empty
4        empty
         ...  
33674    empty
33675    empty
33676    empty
33677    empty
33678    empty
Length: 33679, dtype: object